In [ ]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/lena.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [ ]:
import sys ; sys.path.append('../')
from lena.luenberger_observer import LuenbergerObserver
from lena.system import VanDerPol, RevDuffing
from lena.learner import Learner

In [ ]:
system = VanDerPol()
data = system.generate_mesh([-3.,3.], 30000)

observer = LuenbergerObserver(2, 1)
observer.set_dynamics(system)

learner = Learner(observer,data)
learner.train()

In [ ]:
import torch 
tsim = (0, 20)
dt = 1e-2
tq, simulation = system.simulate(torch.tensor([[1.],[1.]]), tsim, dt)
measurement = observer.h(
        simulation[:, :observer.dim_x, 0].T).T
y = torch.cat((tq.unsqueeze(1), measurement), dim=1)

# Predict from measurment
estimation = observer.predict(y, tsim, dt)

In [ ]:
import matplotlib.pyplot as plt

# Plot truth ground data and estimation
plt.plot(tq, simulation[:,:,0])
plt.plot(tq, estimation.detach().numpy())
plt.show()